In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import seaborn as sb
sb.set_style('darkgrid')

from glob import glob
import matplotlib.pyplot as plt
import os
import random
# import cv2

import gzip
import nibabel as nb

In [4]:
MriFiles = glob('/Volumes/DISK_IMG/10*/*T1w.nii.gz')
SosFiles = glob('/Volumes/DISK_IMG/10*/*Vp.nii.gz')

MriIm = [] # array of full images containing header and data of MRIs
MriHdr = [] # array of only the headers of MRIs
MriData = [] # array of only the data of MRIs

# similarly, for speed of sound maps
SosIm = []
SosHdr = []
SosData = []
for i in range(len(MriFiles)):
    MriIm.append(nb.load(MriFiles[i]))
    MriHdr.append(MriIm[i].header)
    MriData.append(MriIm[i].get_fdata())

    SosIm.append(nb.load(SosFiles[i]))
    SosHdr.append(SosIm[i].header)
    SosData.append(SosIm[i].get_fdata())

In [5]:
# put all the scans of all MRIs as members of a dataframe so that each cell of each row is a whole image
MriDf = pd.DataFrame()
for scan in range(len(MriData)):
    Mri = MriData[scan]
    for slice in range(len(Mri[1,1,:])):
        temp = {'img' : [Mri[:,:,slice]]}
        tempDf = pd.DataFrame.from_records(temp)
        MriDf = MriDf.append(tempDf)
print("Length of MRI dataset: " + str(MriDf.shape[0]))
# same for SoS
SosDf = pd.DataFrame()
for scan in range(len(SosData)):
    Sos = SosData[scan]
    for slice in range(len(Sos[1,1,:])):
        temp = {'img' : [Sos[:,:,slice]]}
        tempDf = pd.DataFrame.from_records(temp)
        SosDf = SosDf.append(tempDf)
print("Length of SoS dataset: " + str(SosDf.shape[0]))

Length of MRI dataset: 1600
Length of SoS dataset: 1600


In [6]:
# must vectorize each cell of the df (each (n,m) matrix) so structure is:
# X has form (n.m, 2) (col of 1s and col of values)
# y has form (n.m, 1)
vec_MriDf = pd.DataFrame()
for row in range(MriDf.shape[0]):
    temp = pd.DataFrame({'MRI' : np.array([MriDf.iloc[row][0].flatten()]).tolist()})
    vec_MriDf = pd.concat([vec_MriDf,temp])

In [7]:
vec_SosDf = pd.DataFrame()
for row in range(SosDf.shape[0]):
    temp = pd.DataFrame({'Sos' : np.array([SosDf.iloc[row][0].flatten()]).tolist()})
    vec_SosDf = pd.concat([vec_SosDf,temp])

In [5]:
"""
making it so each column is a pixel. eg.
       MRI              becomes              0    1    2
 0  [a, b, c]                           0    a    b    c
 1  [d, e, f]                           1    d    e    f
 2  [g, h, i]                           2    g    h    i
 3  [j, k, l]                           3    j    k    l
       ...                                       ...

https://stackoverflow.com/questions/35491274/split-a-pandas-column-of-lists-into-multiple-columns


"""
# MriDf2 = pd.DataFrame(np.nan, index=[x for x in range(vec_MriDf.shape[0])], columns=[x for x in range(len(vec_MriDf.iloc[0][0]))]) #1600x81920
# for slice in range(5): #vec_MriDf.shape[0]
#       for pixel in range(81920): #len(vec_MriDf.iloc[0][0]
#             MriDf2.iloc[slice][pixel] = vec_MriDf.iloc[slice][0][pixel]

In [ ]:
# takes 2.8s for 10 rows. 2m15.3s for all 1600 rows omg
MriDf2 = pd.DataFrame()
MriDf2 = pd.DataFrame(vec_MriDf['MRI'].tolist(), columns=[x for x in range(81920)])

In [ ]:
# same for SoS
SosDf2 = pd.DataFrame()
SosDf2 = pd.DataFrame(vec_SosDf['Sos'].tolist(), columns=[x for x in range(81920)])

In [8]:
MriDf2

,0,1,2,3,4,5,6,7,8,9,...,81910,81911,81912,81913,81914,81915,81916,81917,81918,81919
0,32.055248,72.657677,19.232569,10.684505,40.599922,48.077175,57.690296,16.024361,11.750614,21.363602,...,31.764345,33.663982,18.262636,43.229057,24.964970,35.513618,33.584423,30.699942,16.307798,0.0
1,32.053810,45.943233,26.710699,28.846865,41.666458,29.913332,47.004768,12.818902,14.954640,34.180183,...,9.625598,14.427481,30.758255,24.016249,32.646648,42.232609,5.757357,25.903198,22.063597,0.0
2,38.461998,54.487164,16.025345,12.819966,35.253857,35.252613,37.387615,17.090704,18.157961,13.884743,...,41.390320,20.198601,8.650816,29.780350,16.323437,63.349358,33.584820,15.350153,59.476212,0.0
3,19.229296,53.414051,41.662193,18.160004,51.273762,39.522118,41.656742,6.408435,29.904516,52.330025,...,25.989485,37.512005,30.758728,32.662617,8.641900,15.357566,38.383018,36.456970,42.209339,0.0
4,11.749948,24.567768,25.635456,21.362354,17.089365,27.769218,35.244087,40.582218,25.629560,39.510044,...,40.428509,14.427853,9.612211,16.331497,24.005562,42.233818,28.787619,24.944551,15.349041,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,38.525219,6.916131,31.625824,57.344063,32.642582,28.702501,11.884878,16.849777,1.984021,12.908306,...,21.083086,23.080332,30.092573,31.084820,48.117317,54.119350,72.146484,66.126541,48.089214,0.0
1596,19.750137,17.778591,52.363361,32.616272,29.665512,43.534607,40.593616,48.551422,57.518185,50.624092,...,39.141003,30.094542,52.142696,26.062250,40.084084,61.113945,28.047380,16.025198,60.090969,0.0
1597,12.834820,17.774757,28.645477,37.550041,13.840928,24.730259,7.919007,33.681526,68.412178,39.696697,...,39.132130,47.137409,30.075487,33.071484,21.039351,34.055748,43.062939,76.102325,42.054081,0.0
1598,18.756559,23.697119,46.420422,14.820788,27.678877,5.934624,42.560093,24.763174,53.534237,17.861607,...,17.055660,19.053385,14.033630,30.061562,13.022876,46.070168,9.012145,31.038195,20.023466,0.0


In [9]:
MriDf2.to_pickle("/Volumes/DISK_IMG/MriDf.pkl")
SosDf2.to_pickle("/Volumes/DISK_IMG/SosDf.pkl")

In [10]:
df = pd.DataFrame()
df = pd.concat([MriDf2, SosDf2], axis=1)

In [ ]:
df

Error: Kernel is dead

In [2]:
MriDf = pd.read_pickle("/Volumes/DISK_IMG/MriDf.pkl")

In [4]:
SosDf = pd.read_pickle("/Volumes/DISK_IMG/SosDf.pkl")

In [5]:
df = pd.DataFrame()
df = pd.concat([MriDf, SosDf], axis=1)

In [6]:
df.to_pickle("/Volumes/DISK_IMG/df.pkl")

In [7]:
# 10 mins, sheesh
# lets do 70% train, 15% test, 15% eval
df_train = pd.DataFrame()
train_indices = []
for i in range(int(0.7*df.shape[0])):
    # get 70% of the set for training from the whole thing randomly
    r = random.randint(0, df.shape[0]-1) 
    train_indices.append(r)
    df_train = df_train.append(df.iloc[r])
print(df_train.shape)
df_train.head()

(1120, 163840)


,0,1,2,3,4,5,6,7,8,9,...,81910,81911,81912,81913,81914,81915,81916,81917,81918,81919
352,44.964645,7.676975,24.128033,42.773342,13.161375,39.485344,40.583576,15.356538,28.520470,4.387957,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
1155,23.285669,36.066898,31.379349,39.458214,22.010838,36.991714,5.765597,10.348711,71.065063,38.834438,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
335,34.362698,39.733799,17.183430,30.073902,19.335539,24.710051,41.906578,19.345119,9.674576,40.857559,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
1517,42.007717,18.001348,45.595970,59.981640,61.164623,39.564186,25.167614,27.552523,31.131062,27.524014,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
1524,10.597143,49.450928,37.674141,69.454910,61.206997,43.544712,44.713871,50.587250,62.337761,23.517797,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0


In [9]:
df_train.to_pickle("/Volumes/DISK_IMG/df_train.pkl")

In [10]:
# get the stuff that isnt in train
# dataframe1[~dataframe1.column1.isin(common.column1)]
df_test = pd.DataFrame()
test_indices = []
while df_test.shape[0] < 0.15*df.shape[0]:
    r = random.randint(0, df.shape[0]-1)

    if(r not in train_indices): 
        test_indices.append(r)
        df_test = df_test.append(df.iloc[r])
print("Number of test images: " + str(df_test.shape[0]))

df_eval = pd.DataFrame()
# for i in range(int(0.15*df.shape[0])):
while df_eval.shape[0] < 0.15*df.shape[0]:
    r = random.randint(0, df.shape[0]-1)

    if(r not in train_indices) and (r not in test_indices) :
        df_eval = df_eval.append(df.iloc[r])
print("Number of eval images: " + str(df_eval.shape[0]))

Number of test images: 240
Number of eval images: 240


In [11]:
df_test.to_pickle("/Volumes/DISK_IMG/df_test.pkl")
df_eval.to_pickle("/Volumes/DISK_IMG/df_eval.pkl")

In [21]:
df_train.iloc[:, [81919+x for x in range(81920)]]

,81919,0,1,2,3,4,5,6,7,8,...,81909,81910,81911,81912,81913,81914,81915,81916,81917,81918
352,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
1155,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
335,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
1517,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
1524,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
243,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
855,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0
234,0.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,...,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0,1480.0


In [1]:
import pandas as pd
import pickle5 as pickle
# the kernel died so i have to load this again :(
df_train = pd.read_pickle("/Volumes/DISK_IMG/df_train.pkl")
df_test = pd.read_pickle("/Volumes/DISK_IMG/df_test.pkl")

ValueError: unsupported pickle protocol: 5

In [8]:
import pickle5 as pickle
df_train = pickle.loads("/Volumes/DISK_IMG/df_train.pkl")

TypeError: a bytes-like object is required, not 'str'

In [2]:
import pandas as pd
import pickle5 as pickle
with open(path_to_protocol5, "rb") as fh:
    df_train = pickle.load("/Volumes/DISK_IMG/df_train.pkl")
    df_test = pickle.load("/Volumes/DISK_IMG/df_test.pkl")

NameError: name 'path_to_protocol5' is not defined

In [3]:
df_train_X = df_train.iloc[:, [x for x in range(81920)]]
df_train_y = df_train.iloc[:, [81920+x for x in range(81920)]]
df_test_X = df_test.iloc[:, [x for x in range(81920)]]
df_test_y = df_test.iloc[:, [81920+x for x in range(81920)]]

In [4]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(df_train_X, df_train_y)

In [ ]:
pred = linear.predict(df_test_X)

Error: Session cannot generate requests